In [8]:
import transformers
import pandas as pd
import os
import shutil

In [14]:
undisputed_best_model = transformers.MBartForConditionalGeneration.from_pretrained(
"ml6team/mbart-large-cc25-cnn-dailymail-nl-finetune"
)

tokenizer = transformers.MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")

In [15]:
def summarize(filename):
    global df
    path = 'data/' + filename
    with open(path, 'r', encoding="utf-8") as f:
        
        number_of_characters = 0
        remove_n = []

        lines = f.readlines()
        for line in lines:
                number_of_characters = number_of_characters + len(line)
                remove_n.append(line.strip())

    joined_lines = ''.join(remove_n)

    


    summarization_pipeline = transformers.pipeline(
        task="summarization",
        model=undisputed_best_model,
        tokenizer=tokenizer,
    )
    summarization_pipeline.model.config.decoder_start_token_id = tokenizer.lang_code_to_id[
        "nl_XX"
    ]
    article = joined_lines  # Dutch
    summary = summarization_pipeline(
        article,
        do_sample=True,
        top_p=0.75,
        top_k=50,
        # num_beams=4,
        min_length=50,
        early_stopping=True,
        truncation=True,
    )[0]["summary_text"]

    
    return summary


# To dataframe (too long to wait)

In [ ]:
# iterate through the folder
'''
def summarize_all(directory):
    # create   df     
    data = {'File': [],
        'Summary': []}
    df = pd.DataFrame(data)

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            print(filename)
            output = summarize(filename)
            print(output)
            values_to_add = {'File': filename, 'Summary': output}
            row_to_add = pd.Series(values_to_add, name=filename)
            print(row_to_add)

            df = df.append(row_to_add)
            print(df.shape)
            continue
        else:
            continue
    
    return df

directory = 'data/'
df = summarize_all(directory)
'''

# To txt files (too many of them)

In [18]:
def summarize_all(directory):
    

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            print(filename)
            output = summarize(filename)
            # print(output)
            
            dest_dir = "summaries/"
            #with open(dest_dir + filename, 'w') as f:
                #f.write(output)
                #print(filename)
            file = open(dest_dir + filename, 'w', encoding="utf-8")
            print(dest_dir)
            file.write(output)
            print(filename)
            file.close()

            move_from = directory + filename
            move_to = "bring_back_later/" + filename
            shutil.move(move_from, move_to)
            continue
        else:
            continue


In [ ]:
directory = 'data/'
summarize_all(directory)